In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import sys
sys.path.append("../..")

In [3]:
import pandas as pd
import numpy as np
import pipit as pp

In [4]:
trace = pp.Trace.from_otf2("../../pipit/tests/data/ping-pong-otf2/")
pandas_df = trace.events[["Timestamp (ns)", "Name", "Event Type", "Process", "Thread"]].copy()
pandas_df["Process"] = pandas_df["Process"].astype(int)
pandas_df["Thread"] = pandas_df["Thread"].astype(int)

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

24/02/23 04:22:51 WARN Utils: Your hostname, FEIG5S9893 resolves to a loopback address: 127.0.1.1; using 172.27.119.109 instead (on interface eth0)
24/02/23 04:22:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/23 04:22:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
df = spark.createDataFrame(pandas_df)
df.show()

+--------------------+--------------------+----------+-------+------+
|      Timestamp (ns)|                Name|Event Type|Process|Thread|
+--------------------+--------------------+----------+-------+------+
|                 0.0|        ProgramBegin|   Instant|      1|     0|
|  30083.086937435106|int main(int, cha...|     Enter|      1|     0|
|   40288.33150186851|            MPI_Init|     Enter|      1|     0|
|   307730.9358165928|        ProgramBegin|   Instant|      0|     0|
|  336979.73374932166|int main(int, cha...|     Enter|      0|     0|
|  346054.77444467926|            MPI_Init|     Enter|      0|     0|
| 1.936431381741584E8|            MPI_Init|     Leave|      0|     0|
| 1.936438354832178E8|            MPI_Init|     Leave|      1|     0|
|1.9365164620379105E8|       MPI_Comm_size|     Enter|      0|     0|
|1.9365271483827707E8|       MPI_Comm_size|     Enter|      1|     0|
|1.9365316300611198E8|       MPI_Comm_size|     Leave|      0|     0|
|1.9365416291198432E

In [10]:
# # Filter rows where "Event Type" is "Enter"
# filtered_df = df.filter(col("Event Type") == "Enter")

# # Group by "Name" and "Process", then sum
# grouped_df = filtered_df.groupby(["Name", "Process"]).sum()

# # Group by "Name" and calculate the mean
# result_df = grouped_df.groupby("Name").mean()

# # Show the result
# result_df.show()

In [11]:
from pyspark.sql import functions as F

In [12]:
F.col("Name") > 5

Column<'(Name > 5)'>

In [14]:
# from pipit.dsl import functions as FF
# FF.col("Name")

In [15]:
import polars as pl

polars_df = pl.from_pandas(pandas_df)

polars_df.filter(pl.col("Process") == 0)

Timestamp (ns),Name,Event Type,Process,Thread
f64,cat,cat,i64,i64
307730.935817,"""ProgramBegin""","""Instant""",0,0
336979.733749,"""int main(int, …","""Enter""",0,0
346054.774445,"""MPI_Init""","""Enter""",0,0
1.9364e8,"""MPI_Init""","""Leave""",0,0
1.9365e8,"""MPI_Comm_size""","""Enter""",0,0
1.9365e8,"""MPI_Comm_size""","""Leave""",0,0
1.9366e8,"""MPI_Comm_rank""","""Enter""",0,0
1.9366e8,"""MPI_Comm_rank""","""Leave""",0,0
1.9367e8,"""MPI_Send""","""Enter""",0,0


In [16]:
F.col("Process")

Column<'Process'>

In [17]:
F.sum(F.col("Process") + F.col("Process"))

Column<'sum((Process + Process))'>

In [18]:
pl.col("Process") + pl.col("Process")

In [19]:
type(pl.col("Process"))

polars.expr.expr.Expr

In [36]:
from pipit.dsl._pandas import PandasInterface
from pipit.dsl.event import Event
from pipit.dsl.expr import Column

ds = PandasInterface()

In [37]:
ds.show()

(Empty dataset)



In [45]:
row = pandas_df.iloc[0]
evt = Event(timestamp=row["Timestamp (ns)"], process=row["Process"], thread=row["Thread"], name=row["Name"], event_type=row["Event Type"])
evt

Event<'0.0 Instant ProgramBegin 1 0'>

In [46]:
ds.push_event(evt)

In [47]:
ds.flush()
ds.show()

+----+-------------+--------------+--------------+-----------+----------+
|    |   timestamp | name         | event_type   |   process |   thread |
|----+-------------+--------------+--------------+-----------+----------|
|  0 |           0 | ProgramBegin | Instant      |         1 |        0 |
|  0 |           0 | ProgramBegin | Instant      |         1 |        0 |
+----+-------------+--------------+--------------+-----------+----------+


In [48]:
for i in range(1, len(pandas_df)):
    row = pandas_df.iloc[i]
    evt = Event(timestamp=row["Timestamp (ns)"], process=row["Process"], thread=row["Thread"], name=row["Name"], event_type=row["Event Type"])
    ds.push_event(evt)

In [49]:
ds.flush()
ds.show()

+-----+------------------+-----------------------+--------------+-----------+----------+
|     |        timestamp | name                  | event_type   |   process |   thread |
|-----+------------------+-----------------------+--------------+-----------+----------|
|   0 |      0           | ProgramBegin          | Instant      |         1 |        0 |
|   0 |      0           | ProgramBegin          | Instant      |         1 |        0 |
|   0 |  30083.1         | int main(int, char**) | Enter        |         1 |        0 |
|   1 |  40288.3         | MPI_Init              | Enter        |         1 |        0 |
|   2 | 307731           | ProgramBegin          | Instant      |         0 |        0 |
|   3 | 336980           | int main(int, char**) | Enter        |         0 |        0 |
|   4 | 346055           | MPI_Init              | Enter        |         0 |        0 |
|   5 |      1.93643e+08 | MPI_Init              | Leave        |         0 |        0 |
|   6 |      1.93644e

In [50]:
ds.filter("process == 0").show()

+-----+------------------+-----------------------+--------------+-----------+----------+
|     |        timestamp | name                  | event_type   |   process |   thread |
|-----+------------------+-----------------------+--------------+-----------+----------|
|   2 | 307731           | ProgramBegin          | Instant      |         0 |        0 |
|   3 | 336980           | int main(int, char**) | Enter        |         0 |        0 |
|   4 | 346055           | MPI_Init              | Enter        |         0 |        0 |
|   5 |      1.93643e+08 | MPI_Init              | Leave        |         0 |        0 |
|   7 |      1.93652e+08 | MPI_Comm_size         | Enter        |         0 |        0 |
|   9 |      1.93653e+08 | MPI_Comm_size         | Leave        |         0 |        0 |
|  11 |      1.93656e+08 | MPI_Comm_rank         | Enter        |         0 |        0 |
|  13 |      1.93657e+08 | MPI_Comm_rank         | Leave        |         0 |        0 |
|  15 |      1.93668e